In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

In [ ]:
tf.__version__

### Using Energy Dataset from UCI

In [ ]:
file = '../input/eergy-efficiency-dataset/ENB2012_data.csv'

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.head()

### In the above sample data, we have inputs from X1,X2..X8 and we have two outputs Y1 and Y2.In the Multi Head Networks we essentially will have multiple outputs with the same sets of inputs

In [ ]:
# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

In [ ]:
train.head()

In [ ]:
train_stats

We have data with different distributions and data ranges are differnt. So, obvious step here is to make the data normalize/standardise

In [ ]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2

In [ ]:
# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

In [ ]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [ ]:
# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

# Build the Model

## We have three stages in building the model in functional API
1. Declare Inputs
2. Core Model Architecture (except inuput)
3. Model Object with inptuts and  Outputs

### Declare Input Layer

In [ ]:
input = Input(shape=(len(train.columns),))

### Core Model Architecture

In [ ]:
L1 = Dense(units='128', activation='relu')(input)
L2 = Dense(units='128', activation='relu')(L1)
L3 = Dense(units='64', activation='relu')(L2)

# Y1 output will be fed directly from the second dense
y1_output = Dense(units='1', name='y1_output')(L2)

# Y2 output will come via the third dense
y2_output = Dense(units='1', name='y2_output')(L3)

You can observe y1_output & y2_output has differnt no of intermediate layers, this is where the functional API is useful. And, each layers input is seperately mentioned as a parameter in the paranthesis after declaring the layer. For example, L3 can fed from L1 also just by changing the L1 in the paranthesis.

### Declaring Model Object

In [ ]:
model = Model(inputs=input, outputs=[y1_output, y2_output])

If you have multiple inputs or outputs just pass them as a list to the model object

### See the Architecture

In [ ]:
model.summary()

## Compile the model

In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

## Train the model

In [ ]:
history = model.fit(norm_train_X, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test_X, test_Y))

### Model Evaluation

In [ ]:
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

In [ ]:
def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)

### You can observe the true and predicted values are very close and the loss is very minimal 

### These are plots of loss from epoch to epoch during the trainnig process

Credits to Laurence Moroney,Coursera